#### Doublet Detection 

This notebook uses [Doublet Detection](https://github.com/JonathanShor/DoubletDetection) to predict doublets.

In [1]:
import numpy as np
import pandas as pd
import doubletdetection
import scanpy as sc
import matplotlib.pyplot as plt

In [2]:
# Getting train data

df_train = pd.read_csv("../train/training.csv", header=None)
df_train.columns = ['UMI', 'doublet']
df_train

,UMI,doublet
0,GCTCTGTCAATGGATA,1
1,GATGAGGGTACGAAAT,0
2,AGGCCACGTACAGCAG,0
3,ACGCCGAGTCACACGC,0
4,TCTTCGGAGGCTAGCA,0
...,...,...
995,ACTGCTCCACTCGACG,0
996,TGGTTAGGTAAACGCG,0
997,TTCTCAATCAGTACGT,1
998,TGAGCCGGTCTCTTAT,0


In [3]:
# Get the counts of doublets and singlets 

df_train['doublet'].value_counts()

doublet
0    855
1    145
Name: count, dtype: int64

In [4]:
# read the test data 

adata = sc.read_h5ad('../test_data/sce.h5ad') 

In [5]:
# get expected doublet number

expected_n = int(len(adata.obs) * (145/1000))

print(expected_n)

3183


In [6]:
# actual expected doublet number: 2892 

In [7]:
# referencing past threshold chart to match the expected number of doublets 

# file location: f"prediction\prediction{n}\threshold_test.pdf" where n is the last prediction

In [114]:
# adjusting thresholds

p_thresh = 1e-10
voter_thresh = 0.62

#### Prediction stage

Use BoostClassifier with the adjusted threshold

In [85]:
clf = doubletdetection.BoostClassifier(
    n_iters=30,
    clustering_algorithm="louvain",
    standard_scaling=True,
    verbose=False,
    #pseudocount=0.1,
    #n_jobs=-1,
)

clf.fit(adata.X)

for thresh in range(7, 13):
    doublets = clf.predict(p_thresh=thresh)
    doublet_score = clf.doublet_score()
    adata.obs[f"doublet_{thresh}"] = doublets
    adata.obs[f"doublet_score_{thresh}"] = doublet_score

  0%|          | 0/30 [00:00<?, ?it/s]

Iteration   1/30

Creating synthetic doublets...
Normalizing...
Running PCA...
Clustering augmented data set...

Found clusters [0, ... 34], with sizes: [1771, 1601, 1566, 1548, 1485, 1452, 1404, 1288, 1272, 1227, 1155, 1018, 1017, 966, 966, 955, 812, 767, 742, 740, 592, 458, 434, 339, 337, 292, 235, 200, 180, 177, 120, 118, 106, 93, 9]

Iteration   2/30

Creating synthetic doublets...
Normalizing...
Running PCA...
Clustering augmented data set...

Found clusters [0, ... 34], with sizes: [1591, 1573, 1532, 1431, 1417, 1316, 1314, 1248, 1231, 1145, 1112, 1061, 1040, 988, 981, 963, 754, 698, 679, 670, 572, 542, 512, 448, 440, 367, 346, 330, 284, 222, 205, 122, 107, 103, 98]

Iteration   3/30

Creating synthetic doublets...
Normalizing...
Running PCA...
Clustering augmented data set...

Found clusters [0, ... 36], with sizes: [1790, 1447, 1387, 1387, 1341, 1224, 1212, 1209, 1192, 1175, 1127, 1122, 1022, 1016, 994, 978, 902, 735, 697, 695, 666, 557, 552, 440, 364, 348, 328, 255, 245, 210, 

In [117]:
# saving the doublets and doublet scores in adata

adata.obs["doublet"] = doublets
adata.obs["doublet_score"] = doublet_score

#### Visualizing results

In [118]:
# setting the prediction round count

pred_count = 5

In [119]:
# preparing for umap

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

KeyboardInterrupt: 

In [89]:
sc.pl.umap(adata, color=["doublet", "doublet_score"], save='umap.pdf')

plt.show()

c:\Users\lunat\miniconda3\envs\biomed\lib\site-packages\scanpy\plotting\_utils.py:314: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()
C:\Users\lunat\AppData\Local\Temp\ipykernel_10436\3813664385.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [120]:
type(adata.obs)

pandas.core.frame.DataFrame

In [105]:
# put the observation doublet results in a new df 

df_detected = adata.obs

In [106]:
df_detected

,doublet,doublet_score
AGAGCGAAGATCTGCT,1.0,3.835109e+01
ACATCAGTCTGACCTC,0.0,2.356285e+01
TTCTCCTGTCCTGCTT,1.0,3.835109e+01
GAAACTCTCATGTAGC,0.0,2.356285e+01
CAGTCCTGTCTAGCCG,0.0,2.356285e+01
...,...,...
TTTATGCTCTTGTACT,0.0,3.713946e-46
ATAACGCCAAGCTGGA,0.0,2.097192e-54
ATTACTCAGTGGACGT,0.0,3.713946e-46
CGTGTCTCAAGCGTAG,0.0,1.617720e-50


In [107]:
# check to see if the detected result matches the expected count

df_detected['doublet'].value_counts()

doublet
0.0    19210
1.0     2744
Name: count, dtype: int64

In [94]:
boxplot = df_detected.boxplot(
    column=['doublet_score'], 
    by=['doublet'],
    #save='../prediction/prediction1/boxplot.pdf'
)

plt.show()
plt.savefig(f"../prediction/prediction{pred_count}/boxplot.png")

C:\Users\lunat\AppData\Local\Temp\ipykernel_10436\988335967.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [95]:
sc.pl.violin(adata, "doublet_score")
plt.savefig(f"../prediction/prediction{pred_count}/violin.png")

c:\Users\lunat\miniconda3\envs\biomed\lib\site-packages\scanpy\plotting\_utils.py:314: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.show()


In [121]:
# get a doublet-only and singlet-only df from train data

doublets_UMI = df_train[df_train['doublet']==1]['UMI']
singlets_UMI = df_train[df_train['doublet']==0]['UMI']

doublet_df = df_detected[df_detected.index.isin(doublets_UMI)]

singlet_df = df_detected[df_detected.index.isin(singlets_UMI)]
singlet_df

,doublet,doublet_score
AAGTCTGAGCGTCTAT,0.0,2.444479e+01
AGATTGCGTTTGGGCC,0.0,3.457425e+01
CATGACATCTATCGCC,0.0,2.356285e+01
GTCCTCATCCTTGGTC,1.0,3.835109e+01
AGGCCGTTCACATACG,0.0,2.356285e+01
...,...,...
GCTTGAACATGCCACG,0.0,1.086942e-49
GGTGAAGCATTTCAGG,0.0,1.086942e-49
TGTCCCATCCACGACG,0.0,3.713946e-46
GCGCAGTGTTGTGGAG,0.0,3.713946e-46


In [122]:
df_train

,UMI,doublet
0,GCTCTGTCAATGGATA,1
1,GATGAGGGTACGAAAT,0
2,AGGCCACGTACAGCAG,0
3,ACGCCGAGTCACACGC,0
4,TCTTCGGAGGCTAGCA,0
...,...,...
995,ACTGCTCCACTCGACG,0
996,TGGTTAGGTAAACGCG,0
997,TTCTCAATCAGTACGT,1
998,TGAGCCGGTCTCTTAT,0


In [123]:
df_detected = df_detected.reset_index()
df_detected

,index,doublet,doublet_score
0,AGAGCGAAGATCTGCT,1.0,3.835109e+01
1,ACATCAGTCTGACCTC,0.0,2.356285e+01
2,TTCTCCTGTCCTGCTT,1.0,3.835109e+01
3,GAAACTCTCATGTAGC,0.0,2.356285e+01
4,CAGTCCTGTCTAGCCG,0.0,2.356285e+01
...,...,...,...
21949,TTTATGCTCTTGTACT,0.0,3.713946e-46
21950,ATAACGCCAAGCTGGA,0.0,2.097192e-54
21951,ATTACTCAGTGGACGT,0.0,3.713946e-46
21952,CGTGTCTCAAGCGTAG,0.0,1.617720e-50


In [125]:
df_detected.columns = ['UMI', 'doublet_detected', 'doublet_score']

In [126]:
df_train = df_train.merge(df_detected, on='UMI', how='left')

In [127]:
df_train

,UMI,doublet,doublet_detected,doublet_score
0,GCTCTGTCAATGGATA,1,1.0,2.723823e+02
1,GATGAGGGTACGAAAT,0,1.0,3.724646e+02
2,AGGCCACGTACAGCAG,0,0.0,6.849729e-05
3,ACGCCGAGTCACACGC,0,0.0,4.106492e-23
4,TCTTCGGAGGCTAGCA,0,0.0,1.086942e-49
...,...,...,...,...
995,ACTGCTCCACTCGACG,0,0.0,3.052982e-02
996,TGGTTAGGTAAACGCG,0,1.0,1.400135e+02
997,TTCTCAATCAGTACGT,1,1.0,3.684269e+02
998,TGAGCCGGTCTCTTAT,0,0.0,1.967257e-04


In [129]:
import seaborn as sns

sns.boxplot(data=df_train, x="doublet_score", y="doublet")


<Axes: title={'center': 'Threshold Diagnostics'}, xlabel='doublet_score', ylabel='doublet'>

In [133]:
doublet_df['doublet_score'].describe()

count    1.450000e+02
mean     2.041853e+02
std      2.105833e+02
min      3.713946e-46
25%      8.517580e-01
50%      7.315021e+01
75%      3.724646e+02
max      6.600175e+02
Name: doublet_score, dtype: float64

In [134]:
singlet_df['doublet_score'].describe()

count    8.550000e+02
mean     1.687934e+01
std      6.883730e+01
min      8.827736e-61
25%      7.358281e-21
50%      1.299355e-07
75%      1.435105e-01
max      6.524328e+02
Name: doublet_score, dtype: float64

In [130]:
df_train.to_csv(f'../prediction/prediction{pred_count}/df_train.csv', index = False)

In [109]:
doublet_df['doublet'].value_counts()

doublet
1.0    73
0.0    72
Name: count, dtype: int64

In [110]:
### Figures ###

doubletdetection.plot.convergence(clf, save=f'../prediction/prediction{pred_count}/convergence_test.pdf', show=False, p_thresh=1e-16, voter_thresh=0.5)
doubletdetection.plot.threshold(clf, save=f'../prediction/prediction{pred_count}/threshold_test.pdf', show=False, p_step=6)

<Figure size 600x600 with 2 Axes>

In [111]:
df_detected['doublet'] = df_detected['doublet'].astype(int)

In [112]:
df_detected['doublet'].to_csv(f'../prediction/prediction{pred_count}/prediction.csv', header=None)

In [113]:
### Make summary of singlets and doublets and write to file ###
summary = pd.DataFrame(df_detected['doublet'].value_counts())
summary.index.name = 'Classification'
summary.reset_index(inplace=True)
summary = summary.rename({'DoubletDetection_DropletType': 'Droplet N'}, axis=1)

summary.to_csv(f'../prediction/prediction{pred_count}/DoubletDetection_summary.tsv', sep = "\t", index = False)